# Clinical trial

[This tutorial](https://dotnet.github.io/infer/userguide/Clinical%20trial%20tutorial.html) shows how to do Bayesian model selection in Infer.NET to determine if a new medical treatment is effective. We will construct two models, corresponding to an effective or ineffective treatment, and use model selection to determine the posterior probability of each, given some fictional clinical trial data.

This has been updated in Oct 2018 (post Infer.NET open source launch!) to use: 
* the Paket dependency manager to get the package(s) from nuget
* Paket to get the `FSharpWrapper` from github
* the revised namespace `Microsoft.ML.Probabilistic` (was `MicrosoftResearch.Infer`)

[`FSharpWrapper` tutorial examples](https://dotnet.github.io/infer/userguide/FSharp%20Wrapper.html) include:
* TwoCoins
* TruncatedGaussian
* GaussianRanges
* ClinicalTrial
* BayesPoint
* MixtureGaussians

### Links
http://infernet.azurewebsites.net/default.aspx    
https://dotnet.github.io/infer/userguide/Clinical%20trial.html   
https://dotnet.github.io/infer/userguide/Clinical%20trial%20tutorial.html   

## Load & open packages

https://fsprojects.github.io/Paket/

Packages in F# can be installed via `nuget.org` using the Paket dependency manager.
To install packages from NuGet you should first load the Packet package manager:

In [1]:
#load "Paket.fsx"

You can then have Paket install the packages:

In [21]:
Paket.Package
  [   // "Infer.NET" (pre Oct 2018 namespace)
      "Microsoft.ML.Probabilistic"
      "Microsoft.ML.Probabilistic.Compiler"
      //"Microsoft.ML.Probabilistic.FSharp"  // Not on nuget, yet?
      "NETStandard.Library" // Necessary??
  ]

In [22]:
#load "Paket.Generated.Refs.fsx"  // Do we need this?

### `FSharpWrapper`

See  https://github.com/dotnet/infer/blob/master/src/FSharpWrapper/FSharpWrapper.fs   
https://github.com/fsprojects/IfSharp/issues/146 (Thanks for help, Colin Gravill!)   
https://fsprojects.github.io/Paket/github-dependencies.html

In [5]:
Paket.GitHub ["dotnet/infer src/FSharpWrapper/FSharpWrapper.fs"]

In [4]:
// #load "/home/nbuser/IfSharp/bin/paket-files/github/dotnet/infer/src/FSharpWrapper/FSharpWrapper.fs" // Run later.

### Check `paket` files

In [9]:
open System.IO

Project dependencies file:

In [ ]:
File.ReadAllText(@"../../IfSharp/bin/paket.dependencies")

The `paket install` command will analyze your dependencies and automatically generate a `paket.lock` file:

In [ ]:
File.ReadAllText(@"../../IfSharp/bin/paket.lock")

### Assembly search paths and references

In [7]:
#I "/home/nbuser/IfSharp/bin/packages/Microsoft.ML.Probabilistic/lib/netstandard2.0"
#I "/home/nbuser/IfSharp/bin/packages/Microsoft.ML.Probabilistic/lib/netstandard2.0"
#I "/home/nbuser/IfSharp/bin/packages/Microsoft.ML.Probabilistic.Compiler/lib/netstandard2.0"

In [8]:
#r "Microsoft.ML.Probabilistic"
#r "Microsoft.ML.Probabilistic.Compiler"
#r "netstandard"

### Import declarations for modules or namespaces 

In [10]:
open System

In [14]:
open Microsoft.ML.Probabilistic  
open Microsoft.ML.Probabilistic.Algorithms 
open Microsoft.ML.Probabilistic.Models  
open Microsoft.ML.Probabilistic.Distributions  
open Microsoft.ML.Probabilistic.Factors  
open Microsoft.ML.Probabilistic.Math

In [12]:
#load "/home/nbuser/IfSharp/bin/paket-files/github/dotnet/infer/src/FSharpWrapper/FSharpWrapper.fs"

In [13]:
open Microsoft.ML.Probabilistic.FSharp

## Infer.NET: F# script for clinical trial example  

### Data from a clinical trial  

In [15]:
let controlGroup = Variable.Observed<bool>([|false; false; true; false; false|])
let treatedGroup = Variable.Observed<bool>([|true; false; true; true; true |])
let i = controlGroup.Range
let j = treatedGroup.Range

### Prior on being an effective treatment

In [16]:
let isEffective = Variable.Bernoulli(0.5).Named("isEffective");
let probIfTreated = ref (Variable.New<float>())
let probIfControl = ref (Variable.New<float>())

#### `If` block

[Imperative Statement Blocks in F#](https://dotnet.github.io/infer/userguide/Imperative%20Statement%20Blocks%20in%20FSharp.html)     
[Branching on variables to create mixture models](https://dotnet.github.io/infer/userguide/Branching%20on%20variables%20to%20create%20mixture%20models.html)  

In [17]:
let f1 (vb1: Variable<bool>) = 
    probIfControl := Variable.Beta(1.0, 1.0).Named("probIfControl") 
    let controlGroup = Variable.AssignVariableArray 
                           controlGroup i (fun i ->Variable.Bernoulli(!probIfControl))
    probIfTreated := Variable.Beta(1.0, 1.0).Named("probIfTreated")
    let treatedGroup = Variable.AssignVariableArray 
                           treatedGroup j (fun j ->Variable.Bernoulli(!probIfTreated))
    ()

#### `IfNot` block function

In [18]:
let f2 (vb2: Variable<bool>) = 
    let probAll = Variable.Beta(1.0, 1.0).Named("probAll") 
    let controlGroup = Variable.AssignVariableArray
                           controlGroup i (fun i ->Variable.Bernoulli(probAll))
    let treatedGroup = Variable.AssignVariableArray
                           treatedGroup j (fun j ->Variable.Bernoulli(probAll))
    ()

### The model

In [19]:
Variable.IfBlock isEffective f1 f2

### The inference

In [20]:
let ie = InferenceEngine()
printfn "Probability treatment has an effect = %A" (ie.Infer<Bernoulli>(isEffective))
let probIfTreatedPost = ie.Infer<Beta>(!probIfTreated)
let probIfControlPost = ie.Infer<Beta>(!probIfControl)
printfn "Probability of good outcome if given treatment = %A" (probIfTreatedPost.GetMean())
printfn "Probability of good outcome if control = %A" (probIfControlPost.GetMean())

Compiling model...done.
Probability treatment has an effect = Bernoulli(0.7549)
Probability of good outcome if given treatment = 0.7142857143
Probability of good outcome if control = 0.2857142857
